# Instructions on how to prepare the model used in the following publication:
(Under Review) B. Pickering and R. Choudhary, ‘Out-of-Sample, Out of Mind: Quantifying Resilience in Energy Systems with Out-of-Sample Testing’, Energy, 2020

In [ ]:
## Import relevant packages
import os

import xarray as xr
import pandas as pd
import numpy as np

import calliope

## internal package which includes functions for scenario reduction
import utils

os.chdir('model')

## Mean model

In [ ]:
# Mean model can be built directly, with clustering applied within Calliope
mean_model = calliope.Model('model.yaml')

## Individual scenario

In [ ]:
## Load a scenario, e.g. scenario 1, into the model

# Note: CSV files are stored in compressed format (.csv.gz), to minmise the size of the repository.
# Pandas should be able to load this without any issues.

def get_scenario_model(scenario, probability=None):
    
    if isinstance(scenario, list): # we can provide a list of scenarios, for a multi-scenario Calliope model
        scenario = {i: 1 / len(scenario) for i in scenario}
        
    if isinstance(scenario, dict): # a scenario dict gives `scenario_number:probability` as `key:value` pair
        calliope.AttrDict({
            int(k): {
                'techs': {
                    'demand_cooling': {
                        'constraints': {
                            'resource': 'file=scenarios/demand_cooling_{}.csv.gz'.format(k)
                        }
                    }, 
                    'demand_electricity': {
                        'constraints': {
                            'resource': 'file=scenarios/demand_electricity_{}.csv.gz'.format(k)
                        }
                    }
                },
                'model': {'probability': v}}
            for k, v in scenario.items()
        }).to_yaml('scenarios.yaml')
        
        scenario_model = calliope.Model(
            'model.yaml', override_file='overrides.yaml:dmuu', 
            scenario_file='scenarios.yaml:{}'.format(','.join([str(i) for i in scenario]))
        )
        
    else:
        overrides = {}
        for energy in ['cooling', 'electricity']:
            key = 'techs.demand_{}.constraints.resource'.format(energy)
            overrides[key] = 'file=scenarios/demand_{}_{}.csv.gz'.format(energy, scenario)

        scenario_model = calliope.Model('model.yaml', override_dict=overrides)
            
    return scenario_model

#scenario_model = get_scenario_model(scenario=1)

## Scenario reduction

In [ ]:
# To undertake scenario reduction, run all scenarios independently, then load them into one xarray Dataset:
all_scenarios = xr.concat(
    [xr.open_dataset('path_to_scenario_model_{}'.format(i) for i in range(500))], 
    dim=pd.Index(data=[i for i in range(500)], name='scenarios'),
    data_vars='different'
)

# Then run scenario reduction
reduced_scenarios = utils.get_reduced_scenarios(all_scenarios.cost.values, 16)
reduced_scenarios_df = utils.get_redistributed_probabilities(all_scenarios.cost.values, reduced_scenarios)

## Scenario optimisation

In [ ]:
# Create multi-scenario model, for scenario optimisation
# Load the reduced scenarios, any one of: 'reduced_scenarios_carbon.csv', 'reduced_scenarios_CoC.csv', 'reduced_scenarios_monetary.csv'
scenarios = pd.read_csv('reduced_scenarios_carbon.csv', header=0, index_col=0)
scenario_model = get_scenario_model({int(i): scenarios.loc[i].probability for i in scenarios.reduced_scenario.unique()})

# Add SO-related attributes
scenario_model._model_data.attrs['run.mode'] = 'scenario_plan'
scenario_model._model_data.attrs['run.alpha'] = '1'
scenario_model._model_data.attrs['run.beta'] = '0'

# Edit solver, if necessary
#scenario_model._model_data.attrs['run.solver'] = ''

# Recommended to save this to file, for running on a remote cluster:
#scenario_model.to_netcdf('scenario_model.nc')

# If you want to run right here:
#scenario_model.run()